<a href="https://colab.research.google.com/github/30-na/COVID-19_community_transmission_infection_prevalence/blob/main/compute_rt_LCM_counties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load rpy2
# !pip install rpy2==3.0.0
%load_ext rpy2.ipython



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/rpy2/robjects/pandas2ri.py:15: FutureWarning: pandas.core.index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [ ]:
%%R
if (!require(EpiNow2)) {
  remotes::install_github("epiforecasts/EpiNow2", dependencies = TRUE)
}else {
  print("EpiNow2 is allready installed")
}

In [ ]:
%%R
getwd()
# Connect to GitHub
repo_url <- "https://github.com/30-na/COVID-19_community_transmission_infection_prevalence"
dest_dir <- "COVID-19_community_transmission_infection_prevalence"

# Set the working directory to the "content" folder
setwd("/content")

if (file.exists(dest_dir)) {
  cat("The repository already exists. Updating with git pull...\n")
  setwd(dest_dir)
  system("git pull")
} else {
  cat(paste0("The repository doesn't exist. Cloning from ", repo_url, "...\n"))
  system(paste0("git clone ", repo_url, " ", dest_dir))
  setwd(dest_dir)
}

The repository doesn't exist. Cloning from https://github.com/30-na/COVID-19_community_transmission_infection_prevalence...


In [ ]:
%%R
getwd()
# load library
library(dplyr)
library(EpiNow2)
library(data.table)

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union


Attaching package: ‘data.table’



    between, first, last




In [ ]:
%%R
# load NewCase data county level

load("ColabData/counties.transmission.newCase.RDA")
head(counties.transmission.newCase)
# 1. define Targets
target.NCHS = "Large central metro"
names(counties.transmission.newCase)
# set number of cores to use fitting the model
options(mc.cores = 4)
# no benefit on runtime if cores > chains which is set to 4 by default

# literature distributions 
generation_time <- get_generation_time(disease = "SARS-CoV-2", source = "ganyani", fixed = TRUE)
incubation_period <- get_incubation_period(disease = "SARS-CoV-2", source = "lauer", fixed = TRUE)

# define reporting delay as lognormal with mean of 4 days and sd of 1 day in absence of
# evidence. If data on onset -> report then can use estimate_delay to estimate the delay
reporting_delay <- list(mean = convert_to_logmean(4, 1),
                        mean_sd = 0.1,
                        sd = convert_to_logsd(4, 1),
                        sd_sd = 0.1,
                        max = 15)

# filter list of Large Centeral Metro Counties
LCM_counties = counties.transmission.newCase %>%
filter(
UR_category == target.NCHS
) %>%
  mutate(
  state_fips = paste(state, fips_code, county_name, sep="_")
  )
  names(LCM_counties)

[1] "state"                        "fips_code"                   
[3] "county_name"                  "date"                        
[5] "confirm"                      "UR_code"                     
[7] "UR_category"                  "community_transmission_level"
[9] "state_fips"                  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%R
# list of target counties 
target.counties.list = unique(LCM_counties$state_fips)

# initial dataframe
#original.columns = c("date", "rt.mean", "county")
#LCM_counties_rt = data.frame(matrix(nrow = 0, ncol = length(original.columns)))   
#colnames(LCM_counties_rt) = LCM_counties_rt


for(i in 1:length(target.counties.list)){
  # set target county
  target.county = target.counties.list[i]
  
  # set confirm case for the target county
  reported_cases = LCM_counties %>%
  dplyr::filter(
    state_fips == target.county
  ) %>%
  dplyr::select(
    date,
    confirm
  )%>%
  dplyr::arrange(
    date
    )
    
  # run epinow
out <- epinow(reported_cases = reported_cases, 
              generation_time = generation_time,
              delays = delay_opts(incubation_period, reporting_delay),
              rt = rt_opts(prior = list(mean = 1.5, sd = 0.5)),
              # here we define the quality of the gaussian process approximation
              # if the fit to data appears poor try increasing basis_prop and
              # potentially the boundary_scale (see ?gp_opts for details)
              # though this will likely increase runtimes.
              gp = gp_opts(basis_prop = 0.2),
              # in some instances stan chains can get stuck when estimating in 
              # these instances consider using the future fitting mode by passing 
              # `future = TRUE, max_execution_time = 60 * 30` to stan_opts and calling 
              # `future::plan("multiprocess")` prior to running epinow this will time out
              # chains after 30 minutes but still return results from completed chains
              stan = stan_opts(),
              horizon = 14, 
              #target_folder = "results",
              #logs = file.path("logs", Sys.Date()),
              #return_output = TRUE, 
              verbose = TRUE)

# make a data frame
r = (out[["estimates"]][["summarised"]]$variable) == "R"
rt.mean = out[["estimates"]][["summarised"]]$mean[r]
date = out[["estimates"]][["summarised"]]$date[r]

rt.df = data.frame(
  date = as.Date(date, "%Y-%m-%d"),
  rt.mean = rt.mean,
  county = target.counties.list[i]
)

# merged all data
#LCM_counties_rt = rbind(
#    LCM_counties_rt,
#    rt.df
#  )

# save file to github
save(rt.df, file = paste0("/content/drive/My Drive/ColabFiles/", target.counties.list[i], "_rt.rda" ))
}